In [14]:
#Расчёт Пластинчато-трубчатого ТО (корридорный шаг)
import matplotlib.pyplot as pt
import numpy as n
import math as m
import json, CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI as p
from scipy.optimize import root
import math

#исходные данные:
G1 = 298
T11 = 50
T12 = 37
G2 = 298
T21 = 32
P1 = 243000
P2 = 3300000

R236ea = 'REFPROP::R236ea'
s=50

H11=p('H','T', T11+273.15, 'P', P1, R236ea)
H21=p('H','T', T21+273.15, 'P', P2, R236ea)
H12 = p('H','T', T12+273.15, 'P', P1, R236ea)
step = (H11-H12)/s

t1  = n.zeros(s+1)
t2  = n.zeros(s+1)
Q   = n.zeros(s+1)
h11 = H11
h21 = H21
for i in range(s+1):
    t1[i] = p('T','H', h11, 'P', P1, R236ea)-273.15
    if i < s:
        h12 = h11-step
        dQ = G1*(h11-h12)
        h11 = h12
        Q[i+1] = Q[i]+dQ
for i in range(s+1):
    t2[s-i] = p('T','H', h21, 'P', P2, R236ea)-273.15
    if i < s:
        h22 = h21+(Q[s-i]-Q[s-i-1])/G2
        h21 = h22
T12 = t1[s]
T22 = t2[0]
H12 = h11
H22 = h21
DT=t1-t2
minDT=round(min(DT),5)
Q=Q[s]


Dvnes=0.01
DeltaTube=0.001
Dvnut=Dvnes-2*DeltaTube
step=2.5*Dvnes        
Hchan=0.25*Dvnes
Deltaplate = 0.0005
lambdaw = 20
row=7900
etta=1

T1av = (T11+T12)/2
T2av = (T21+T22)/2

ro1 = p('D','T', T1av+273.15, 'P', P1, R236ea)
ro2 = p('D','T', T2av+273.15, 'P', P2, R236ea)
nu1 = p('V','T', T1av+273.15, 'P', P1, R236ea)/ro1
nu2 = p('V','T', T2av+273.15, 'P', P2, R236ea)/ro2
lambda1 = p('L','T', T1av+273.15, 'P', P1, R236ea)
lambda2 = p('L','T', T2av+273.15, 'P', P2, R236ea)
Pr1 = p('PRANDTL','T', T1av+273.15, 'P', P1, R236ea)
Pr2 = p('PRANDTL','T', T2av+273.15, 'P', P2, R236ea)

dTmax = T11 - T22
dTmin = T12 - T21
dT = (dTmax - dTmin) / (n.log(dTmax / dTmin))



############################################################################################
Z=8
w1= 28.5
w2 = 3
############################################################################################

Re2 = w2*Dvnut/nu2
Nu2 = 0.021*Re2**0.8*Pr2**0.43
def func(x):
    delta = 10**-5
    left = x
    right = 1/(2*n.log10((2.51/(Re2*(x)**(1/2)))+(delta/3.7)))**2
    return left-right
sol = root(func, 0.01)
f2=float(sol.x)
alpha2 = Nu2*lambda2/Dvnut
Re1 = w1*Dvnes/nu1
Nu1 = 0.195*Re1**0.65*Pr1**(-1/3)
f1 = 1.7*Re1**(-0.5)
alpha1 = Nu1*lambda1/Dvnes
KF = Q/dT

F_vneshn = 4*(((2**0.5)*step/2)**2 - math.pi*(Dvnes**2)/4) + 2*Hchan* math.pi*Dvnes
F_vntr = math.pi*(Dvnes-2*DeltaTube)*2*(Hchan+Deltaplate)
F_trube = math.pi*Dvnes*2*(Hchan+Deltaplate)
koef1 = F_vntr/F_vneshn
koef2 = F_vneshn / F_trube

def func(x):
    Fvnes=x
    a1 = 1/KF
    a2 = 1/(etta*alpha1*Fvnes)
    a3 = ((1/2)*(n.log(Dvnes/Dvnut))*(Dvnes/(lambdaw*(Fvnes/koef2))))
    a4 = 1/(alpha2*Fvnes*koef1)
    return a1-a2-a3-a4
sol = root(func, 0.002)
x=sol.x
Fvnes=float(x)
Nsumm = m.ceil(Fvnes/(F_vneshn)/Z)
S2 = G2/(ro2*w2)
S1 = G1/(ro1*w1)

#####

NH =  m.ceil(Nsumm*(n.pi*Dvnut**2)/(S2*4))
NW =  m.ceil(S1/(4*NH*Hchan*(step-(Dvnut+2*DeltaTube))))
NL =  m.ceil(4*S2/(NW*n.pi*Dvnut**2))
######    

#Вставить уточнение площади и скоростей
S1_y = 4*NW*NH*Hchan*(step-(Dvnut+2*DeltaTube))
S2_y = NW*NL*n.pi*Dvnut**2/4
w1_y = G1/(ro1*S1_y)
w2_y = G2/(ro2*S2_y)
Nsumm_y = NH*NW*NL
Fvnes_y =Nsumm_y*F_vneshn*Z
print ('F_vnes', Fvnes_y, 'м2')
print ('Nsumm', Nsumm_y*Z)
print ('w1_y', w1_y)
print ('w2_y', w2_y)


W=round(NW*step, 4)
H=round(NH*((Hchan+Deltaplate)*2), 4)
L=round(NL*step*Z, 4)
dP1_y = f1*Z*L/Dvnes*ro1*w1_y**2/2
dP11_y = f1*Z*L/Dvnes*ro1*w1_y**2/2

dP2_y = f2*Z*H/Dvnut*ro2*w2_y**2/2
print('dP1_mtr', dP1_y/1000, 'кПа')
print('dP2_tr', dP2_y/1000, 'кПа')


print('длина', L, 'ширина', W,'высота', H)

Ntryb = NW*NL*Z
Nplate = NH*2*Z
Ltryb = 2* (Hchan+Deltaplate)*NH
Wplate = NW*step
Lplate = NL*step

V = n.pi*(Dvnes**2-Dvnut**2)/4*Ltryb*Ntryb + Lplate*Wplate*Deltaplate*Nplate
# Плотность стали 
ro_stal = 7832 # кг/м3


Massa =   ro_stal * V 
print('V', L*W*H)
print ('Масса', Massa, 'кг')

F_vnes 1526.6173991621054 м2
Nsumm 1396824
w1_y 27.958767646498337
w2_y 2.9305744835994285
dP1_mtr 97.11265348997699 кПа
dP2_tr 79.35360806257422 кПа
длина 7.4 ширина 0.975 высота 0.726
V 5.23809
Масса 8693.368128943392 кг
